In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
# 하이퍼파라미터 지정

num_words = 10000 # imdb에서 볼 단어수
max_len = 200 # 최대 길이 같게
batch_size = 64
embed_dim = 64
hidden_dim = 32
epochs = 5

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# Pytorch 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
class SimpleRNNModel(nn.Module):
  def __init__(self):
    super(SimpleRNNModel, self).__init__()
    self.embedding = nn.Embedding(num_words, embed_dim)
    self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.embedding(x)
    out, _ = self.rnn(x)
    out = out[:, -1, :] # [batch, 32]
    out = self.fc(out)
    return self.sigmoid(out).squeeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleRNNModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
for epoch in range(epochs):
  model.train()
  total_loss = 0
  correct = 0
  total = 9
  for batch_x, batch_y in train_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()
    output = model(batch_x)
    loss = criterion(output,batch_y)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    pred = (output> 0.5).float()
    correct += (pred == batch_y).sum().item()
    total += batch_y.size(0)

  print(f'Epoch: {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}')

Epoch: 1, Loss: 0.6781, Accuracy: 0.5542
Epoch: 2, Loss: 0.6116, Accuracy: 0.6682
Epoch: 3, Loss: 0.5417, Accuracy: 0.7349
Epoch: 4, Loss: 0.5071, Accuracy: 0.7586
Epoch: 5, Loss: 0.5931, Accuracy: 0.6828


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for batch_x, batch_y in test_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    output = model(batch_x)
    pred = (output > 0.5).float()
    correct += (pred == batch_y).sum().item()
    total += batch_y.size(0)

print(f'Test Accuracy: {correct/total:.4f}')

Test Accuracy: 0.6920
